In [1]:
!pip install datasets
!pip install openai==0.28

In [2]:
import openai
import os
from datasets import Dataset
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
from huggingface_hub import login
import torch
import random
import re
import numpy as np

In [3]:
login()

In [10]:
openai.api_key = 'Your-api'
# comment the below line if you use original OPENAI api
openai.api_base = 'https://api.avalai.ir/v1'

In [15]:
# @title prompts
prompt_get_part = f'''

## پرسش
{{question}} \n
این یک پرسش پزشکی است برای طبقه بندی پرسش یکی از موضوعات زیر را انتخاب کنید به گونه ای که بیشترین تطابق را با پرسش داشته باشد

Topic List = [
    "Part 1: The Profession of Medicine",
    "Part 2: Cardinal Manifestations and Presentation of Diseases",
    "Part 3: Pharmacology",
    "Part 4: Oncology and Hematology",
    "Part 5: Infectious Diseases",
    "Part 6: Disorders of the Cardiovascular System",
    "Part 7: Disorders of the Respiratory System",
    "Part 8: Critical Care Medicine",
    "Part 9: Disorders of the Kidney and Urinary Tract",
    "Part 10: Disorders of the Gastrointestinal System",
    "Part 11: Immune-Mediated, Inflammatory, and Rheumatologic Disorders",
    "Part 12: Endocrinology and Metabolism",
    "Part 13: Neurologic Disorders",
    "Part 14: Poisoning, Drug Overdose, and Envenomation",
    "Part 15: Disorders Associated with Environmental Exposures",
    "Part 16: Genes, the Environment, and Disease",
    "Part 17: Global Medicine",
    "Part 18: Aging",
    "Part 19: Consultative Medicine",
    "Part 20: Frontiers"
]
به عنوان پاسخ تنها و تنها یک عدد برگردانید به گونه ای که بیشترین تطابق را با پرسش داشته باشد.
'''

prompt_eval = f'''
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n
این یک پرسش پزشکی است به همراه گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n
'''

prompt_eval_bare = f'''
به پرسش زیر به پاسخ بدهید و تنها از زبان فارسی استفاده کنید!
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

\n
'''

prompt_eval_bare_COT = f'''
به پرسش زیر قدم به قدم فکر کنید و زنجیره افکار (chain of thought) خود برای رسیدن به پاسخ را به طور کامل شرح دهید و تنها یک گزینه را انتخاب کنید.
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n
'''

prompt_eval_bare_fully = f'''
{{question}} \n
{{choices}}
'''

prompt_eval_bare_fully_with_examples = f'''
{{examples}} \n
{{question}} \n
{{choices}}
'''

prompt_eval_with_examples = f'''
## نمونه ها
{{examples}}

Above are examples for medical Q&A.

## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

این یک پرسش پزشکی است به همراه گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n
'''

prompt_eval_context_bare = f'''
{{context}} \n
{{question}} \n
{{choices}}
'''
prompt_eval_with_context = f'''
## Context
{{context}} \n

## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n
این یک متن از یک کتاب مرجع است به همراه یک پرسش پزشکی و گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n '''

prompt_eval_with_context_and_examples = f'''
## نمونه ها
{{examples}}
در بالا نمونه هایی از پرسش پاسخ پزشکی آورده شده است.

## متن
{{context}} \n

## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

این یک پرسش پزشکی است به همراه گزینه هایی که میتوانید انتخاب کنید. به این پرسش قدم به قدم فکر کرده و پاسخ خود را به صورت زیر بیان کنید:
پاسخ تولید شده توسط مدل که قدم به قدم به آن فکر شده. بنابراین پاسخ نهایی [پاسخ نهایی مدل (e.g. A,B,C,D)] میباشد. به عنوان مثال [A]
\n '''

prompt_example = f'''
## پرسش
{{question}} \n

## گزینه ها
{{choices}} \n

## پاسخ
{{answer}} \n
'''

prompt_extraction = f'''
an answer to a multiple choice question is given to you, your task is
to extract the option that has been chosen,
Please provide the extracted answer in the given format,without any additional explanation
# Format
[answer] e.g. [B]
# Answer
{{answer}}
'''

In [5]:
# @title utility
# Set openai key if using gpt4o as engine.
#os.environ['OPENAI_API_KEY'] = "OPEN AI KEY HERE"
def extract_samples(task, numShot, model_prompt):
    questions, answer_choices, correct_answers = task_load(task, 'train')
    example_indexes = random.sample(range(len(questions)), numShot)
    example_list = []
    for i in example_indexes:
        example_list.append(model_prompt.format(question=questions[i], choices=format_choices(answer_choices[i]), answer=correct_answers[i]))
    return example_list

def get_ds_from_df(df,task):
    if task == 'qas':
       df['Question'] = df['Question'].astype(str)
       df['Option1'] = df['Option1'].astype(str)
       df['Option2'] = df['Option2'].astype(str)
       df['Option3'] = df['Option3'].astype(str)
       df['Option4'] = df['Option4'].astype(str)
       df['Expression'] = df['Expression'].astype(str)
       df['Correct_answer'] = df['Correct_answer'].astype(str)
       df['choice_type'] = df['choice_type'].astype(str)
       df['subject_name'] = df['subject_name'].astype(str)
       df['topic_name'] = df['topic_name'].astype(str)
       ds = Dataset.from_pandas(df)
       return ds
    else:
       raise Exception("TASK NOT FOUND")

def append_record_to_excel(file_path, Question, question_choices,
                           correct_answer, model_prompt, AI_answer):
    new_record = {
        'Question': Question,
        'question_choices': question_choices,
        'correct_answer': correct_answer,
        'model_prompt':  model_prompt,
        'AI_answer': AI_answer
    }
    new_record_df = pd.DataFrame([new_record])
    try:
        existing_df = pd.read_excel(file_path)
        updated_df = pd.concat([existing_df, new_record_df], ignore_index=True)
    except FileNotFoundError:
        updated_df = new_record_df

    updated_df.to_excel(file_path, index=False)

def format_choices(choices):
    a = zip(list(choices.keys()), choices.values())
    final_answers = []
    for x,y in a:
        final_answers.append(f'[{x}] : {y}')
    return "\n".join(final_answers)

def format_examples(examples):
    formatted_examples = []
    for row in examples:
        example = f'## Question {row["question"]} \n ## Answer {row["answer"]}'
        formatted_examples.append(example)
    return "\n".join(formatted_examples)

def task_load(task, split):
    if task=="qas":
        df = pd.read_excel(task+'.xlsx')
        ds =get_ds_from_df(df,task)
        questions = [ds[i]['Question'] for i in range(len(ds))]
        answer_choices = [{"A": ds[i]['Option1'], "B": ds[i]['Option2'], "C": ds[i]['Option3'], "D": ds[i]['Option4']} for i in range(len(ds))]
        correct_answers = [chr(int(ds[i]['Correct_answer'])+65) for i in range(len(ds))]
        return questions, answer_choices, correct_answers
    else:
        raise Exception("TASK NOT FOUND")

def filterContext(context):
    end_tag = "</end>"
    if end_tag in context:
        return context.split(end_tag)[0] + end_tag
    return context

def run_inference(content, engine, temp=1, max_tokens_output=200, tokenizer=None, model=None, local=False):
    messages = [{"role": "user", "content": f"{content}"}]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to('cuda:0')
    with torch.no_grad():
         outputs = model.generate(inputs, max_new_tokens=max_tokens_output, do_sample = True, temperature=temp)
         text = tokenizer.batch_decode(outputs)[0]
         answer = re.sub(r'<\|END_OF_TURN_TOKEN\|>$', '', text.split("model")[-1].split("<|CHATBOT_TOKEN|>")[1])
         return answer

def get_unlocal_AI_answer(prompt,m):
    response = openai.ChatCompletion.create(
        model=m,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature = 0.0
    )
    result = response['choices'][0]['message']['content'].strip().lower()
    return result

In [6]:
# @title model setting

print("RUNNING NORMAL IMPLEMENTATION")
ENGINE = "CohereForAI/aya-expanse-8b"
SPLIT = "test"
ENGINE_TEMPERATURE = 1.0
MAX_TOKEN_OUTPUT = 1024
TASK = 'qas'
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = AutoTokenizer.from_pretrained(ENGINE)
model = AutoModelForCausalLM.from_pretrained(ENGINE,torch_dtype=torch.bfloat16).to(device)
model.eval()
## OUTPUT RUN INFO:
print("Model Running: " + ENGINE)

RUNNING NORMAL IMPLEMENTATION


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model Running: CohereForAI/aya-expanse-8b


In [7]:
# @title Load the test
question_list, answer_choices_list, correct_answer_list = task_load(TASK, SPLIT)
print(f"{TASK} loaded succesfully. Now conducting evaluation on {len(question_list)} samples.")

qas loaded succesfully. Now conducting evaluation on 821 samples.


In [21]:
def extract_choice(answer):
    ext = get_unlocal_AI_answer(prompt_extraction.format(answer=answer),'deepseek-chat')
    if ext == '[a]' or '[A]':
       return 'A'
    elif ext == '[b]' or '[B]':
       return 'B'
    elif ext == '[c]' or '[C]':
       return 'C'
    elif ext == '[d]' or '[D]':
       return 'D'
    else:
       return input('answer did not extraxted input manualy\n Answer:' + answer)

def resume_the_test(question_list, answer_choices_list, correct_answer_list,bgn):
    question_list = question_list[bgn:]
    answer_choices_list = answer_choices_list[bgn:]
    correct_answer_list = correct_answer_list[bgn:]
    for i, (question, answer_choices, correct_answer) in tqdm(enumerate(zip(question_list, answer_choices_list, correct_answer_list))):
        prompt = prompt_eval_bare_COT
        model_prompt = prompt.format(question=question, choices=format_choices(answer_choices))
        student_answer = run_inference(model_prompt, ENGINE, ENGINE_TEMPERATURE, MAX_TOKEN_OUTPUT, tokenizer, model, local=True)
        ext = extract_choice(student_answer)
        print(ext)
        print(correct_answer)
        file_path = 'answers.xlsx'
        append_record_to_excel(file_path, question, answer_choices,
                           correct_answer, model_prompt, student_answer)

In [22]:
# Since google colab usage time is limited & this test takes days to complete
#  we need to concatenate the result of many session to get the final result
#   so set the bgn variable to number of question that has been solved in previous sessions
resume_the_test(question_list, answer_choices_list, correct_answer_list,bgn = 0)

1it [00:40, 40.96s/it]

A
B


2it [01:08, 32.98s/it]

A
A


3it [01:41, 33.03s/it]

A
C


4it [02:25, 37.33s/it]

A
D


4it [02:31, 37.93s/it]


KeyboardInterrupt: 